In [ ]:
import earthaccess
import os
import xarray as xr

import warnings
warnings.simplefilter("ignore")

In [ ]:
## After creating an account at Nasa EarthData, I used this script for creating ~/.netrc login file
## Credits: https://cheginit.github.io/til/web_service/netrc.html 
from netrc import netrc
from typing import Dict
from getpass import getpass
from pathlib import Path

def setup_urs_login(urs: str) -> None:
    prompts = {
        "login": "\n".join(
            [
                "Enter NASA EarthData Login Username",
                f"(or create an account at {urs}): ",
            ]
        ),
        "password": "Enter NASA EarthData Login Password: ",
    }

    netrc_file = Path.home().joinpath(".netrc")

    try:
        netrc(netrc_file).authenticators(urs)[1]
    except (FileNotFoundError, TypeError) as exc:
        user_data = {p: getpass(prompt=msg) for p, msg in prompts.items()}

        fmode = "w" if exc.__class__ == FileNotFoundError else "a"
        with open(netrc_file, fmode) as f:
            msg = f"machine {urs}\n"
            msg += "\n".join(f"{p} {m}" for p, m in user_data.items())
            f.write(msg)
    print(f"User-pass info successfully set for {urs}.")

setup_urs_login("https://urs.earthdata.nasa.gov")

# Data retrieval
Dataset: [MODIS/Terra CGF Snow Cover Daily L3 Global 500m SIN Grid, Version 61](https://nsidc.org/data/mod10a1f/versions/61)

More details about code: [earthdata quick start](https://earthaccess.readthedocs.io/en/latest/quick-start/) and [search granules](https://earthaccess.readthedocs.io/en/latest/howto/search-granules/)

In [ ]:
# Step 1: login (uses credentials in .netcr file)
session = earthaccess.login()
session.authenticated

In [ ]:
# Step 2: search for data
bbox_4326 = (10.46750857, 45.69873184, 11.94937569, 46.53633029) # TRENTINO

results = earthaccess.search_data(
    short_name = "MOD10A1F", # dataset
    version = "61",
    #cloud_hosted = True,
    bounding_box = bbox_4326,
    temporal = ("2022-12-06", "2022-12-06"),
    count = -1 # all
)
results


In [ ]:
# Step 3: Download
destination_dir = "../data/MODIS/"
os.makedirs(destination_dir, exist_ok=True)
files = earthaccess.download(results, destination_dir)


In [ ]:
## I use the local file to inspect structure
from pyhdf.SD import SD, SDC

local_file = "../data/MODIS/MOD10A1F.A2022340.h18v04.061.2022351140722.hdf"

# Open the HDF4 file using pyhdf
hdf = SD(local_file, SDC.READ)

# Get the available datasets
datasets = hdf.datasets()
print("Datasets:")
datasets

In [ ]:
dataset_names = list(datasets.keys())

# Then you can use the dataset name to open it
dataset = hdf.select(dataset_names[0])  # Selecting the first dataset
data = dataset.get()               # Get data from the selected dataset
data

In [ ]:
# Alternative Step 3: open (lits of file-like objects)
granules = earthaccess.open(results)
granules

In [ ]:
# use xarray to open the file-like object and extract the desired region
url = granules[0]
ds = xr.open_dataset(url, engine="h5netcdf")
ds